In [1]:
# vacunados region edad
import subprocess
from environs import Env
import pandas as pd

env = Env()
env.read_env()


file_path = 'Datos-COVID19/output/producto78/total_vacunados_sexo_edad.csv'
destination_folder = env('BUCKET_LOCATION')

subprocess.run(['git', 'checkout','master'], cwd='Datos-COVID19')
subprocess.run(['git', 'pull'], cwd='Datos-COVID19')

CompletedProcess(args=['git', 'pull'], returncode=0)

In [2]:
def get_historic_file_git(file_path):

    filename = file_path.split('/')[-1]
    file_path_dir = file_path.replace(filename, '')


    subprocess.run(f'git log --pretty=format:"%h|%ad| %s %d [%an]" --date=iso {filename} > commits.txt', shell=True, cwd=file_path_dir)

    commits_df = pd.read_csv(f'{file_path_dir}/commits.txt', sep='|', names=['commit','datetime','comment'])

    # solo para obtener el header
    df = pd.read_csv(file_path, nrows=0)

    for commit_row in commits_df.iterrows():
        subprocess.run(['git', 'checkout',commit_row[1]['commit']], cwd=file_path_dir)
        
        aux = pd.read_csv(file_path)
        aux['datetime'] = commit_row[1]['datetime']
        
        df = pd.concat([df, aux])

    return df

subprocess.run(['git', 'checkout','master'], cwd='Datos-COVID19')
subprocess.run(['git', 'pull'], cwd='Datos-COVID19')

df_vacunas = get_historic_file_git(file_path)

subprocess.run(['git', 'checkout','master'], cwd='Datos-COVID19')

df_vacunas = df_vacunas[pd.notna(df_vacunas['datetime'])]
df_grouped = df_vacunas.groupby(['Dosis','datetime']).sum()

over_60_cols = set(filter(lambda x: int(x)>=60, df_grouped.columns))
under_60_cols = set(df_grouped.columns).difference(over_60_cols)

df_grouped['60 o mas'] = df_grouped[over_60_cols].sum(axis=1)
df_grouped['Menores de 60'] = df_grouped[under_60_cols].sum(axis=1)

df_grouped = df_grouped[['60 o mas','Menores de 60']]

demo = pd.read_csv('local_postprocessed_data/Chile-2020.csv')

over_60_tot = demo.iloc[12:][['M','F']].sum().sum()
under_60_tot = demo.iloc[0:12][['M','F']].sum().sum()

df_grouped['60 o mas (%)'] = df_grouped['60 o mas']/over_60_tot
df_grouped['Menores de 60 (%)'] = df_grouped['Menores de 60']/under_60_tot

df_grouped.to_csv(f'{destination_folder}/vacunas_diarias_edad_sexo.csv')

In [13]:
# casos genero-edad
src_path = "Datos-COVID19/output/producto16/CasosGeneroEtario.csv"
df_casos = pd.read_csv(src_path)

df_casos = df_casos.groupby('Grupo de edad').sum().transpose()

df_casos_fixed = pd.DataFrame(index=pd.date_range(df_casos.index.min(),df_casos.index.max(), freq="D"))
df_casos_fixed = df_casos_fixed.join(df_casos, how='left')
df_casos_fixed = df_casos_fixed.interpolate()
df_casos_fixed = df_casos_fixed.astype(int)

df_casos_diarios = df_casos_fixed.diff(1)

df_casos_diarios.drop(df_casos_diarios.head(1).index, inplace=True)

over_60_cols = set(['60 - 64 años','65 - 69 años','70 - 74 años','75 - 79 años','80 y más años'])
under_60_cols = set(df_casos_diarios.columns).difference(over_60_cols)

df_casos_diarios['60 o mas'] = df_casos_diarios[over_60_cols].sum(axis=1)
df_casos_diarios['Menores de 60'] = df_casos_diarios[under_60_cols].sum(axis=1)

df_casos_diarios = df_casos_diarios.reset_index().rename(columns={'index':'date'})

df_casos_diarios['Totales'] = df_casos_diarios[['60 o mas','Menores de 60']].sum(axis=1)
df_casos_diarios = df_casos_diarios[['date','60 o mas','Menores de 60','Totales']]

df_casos_diarios.to_csv(f'{destination_folder}/casos_diarios.csv')

In [18]:
df_casos

Grupo de edad,00 - 04 años,05 - 09 años,10 - 14 años,15 - 19 años,20 - 24 años,25 - 29 años,30 - 34 años,35 - 39 años,40 - 44 años,45 - 49 años,50 - 54 años,55 - 59 años,60 - 64 años,65 - 69 años,70 - 74 años,75 - 79 años,80 y más años
2020-03-25,10,6,9,20,68,126,168,151,125,95,61,58,45,24,23,15,8
2020-03-26,12,7,11,23,82,152,204,181,153,115,91,71,60,31,32,16,11
2020-03-27,12,7,12,29,89,178,228,199,172,138,107,83,69,43,36,20,12
2020-03-28,14,10,16,33,106,209,268,234,192,165,139,112,86,52,42,25,20
2020-03-29,15,13,16,37,118,225,283,257,220,187,145,127,102,63,47,28,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-26,19510,19281,24404,41158,78791,110158,107324,86674,73602,73877,71000,66279,49537,33255,24328,17660,24958
2021-03-01,19808,19663,24941,42055,80322,111956,108973,88055,74765,75049,72136,67328,50377,33775,24733,17983,25456
2021-03-05,20259,20154,25682,43308,82405,114518,111549,90112,76576,76804,73736,68843,51531,34565,25258,18364,25986
2021-03-08,20609,20613,26332,44378,84186,116672,113577,91756,77998,78172,75048,70127,52432,35190,25728,18722,26421


In [16]:
# UCI
src_path = "Datos-COVID19/output/producto9/HospitalizadosUCIEtario_T.csv"

df_UCI = pd.read_csv(src_path)
df_UCI.rename(columns={'Grupo de edad':'date'}, inplace=True)
df_UCI['60 o mas'] = df_UCI[['60-69','>=70']].sum(axis=1)
df_UCI['Menores de 60'] = df_UCI[['<=39','40-49','50-59']].sum(axis=1)
df_UCI['Totales'] = df_UCI[['Menores de 60', '60 o mas']].sum(axis=1)
df_UCI.to_csv(f'{destination_folder}/uci_diarios.csv')

In [17]:
# camas UCI

src_path = "Datos-COVID19/output/producto58/Camas_UCI_diarias_t.csv"
camas_uci = pd.read_csv(src_path)


camas_uci_proc = camas_uci[['Region','Total','Total.2']].rename(columns={'Region':'date','Total':'Camas UCI Habilitadas', 'Total.2':'Camas no Covid-19 ocupadas'})

camas_uci_proc.drop(camas_uci_proc.tail(1).index, inplace=True)


camas_uci_proc.to_csv(f'{destination_folder}/camas_uci.csv')

In [13]:
# fallecidos diarios
src_path = "Datos-COVID19/output/producto10/FallecidosEtario_T.csv"

df_muertes_diarias = pd.read_csv(src_path)
df_muertes_diarias.rename(columns={'Grupo de edad':'date'}, inplace=True)
df_muertes_diarias['60 o mas'] = df_muertes_diarias[['60-69','70-79','80-89','>=90']].sum(axis=1)
df_muertes_diarias['Menores de 60'] = df_muertes_diarias[['<=39','40-49','50-59']].sum(axis=1)
df_muertes_diarias['Totales'] = df_muertes_diarias[['Menores de 60', '60 o mas']].sum(axis=1)
df_muertes_diarias.set_index('date', inplace=True)
df_muertes_diarias = df_muertes_diarias.diff(1)
df_muertes_diarias.drop(df_muertes_diarias.head(1).index, inplace=True)
df_muertes_diarias.to_csv('local_postprocessed_data/fallecidos_diarios.csv')
df_muertes_diarias

,<=39,40-49,50-59,60-69,70-79,80-89,>=90,60 o mas,Menores de 60,Totales
date,,,,,,,,,,
2020-04-10,0.0,1.0,1.0,1.0,1.0,2.0,2.0,6.0,2.0,8.0
2020-04-11,1.0,0.0,0.0,2.0,2.0,3.0,0.0,7.0,1.0,8.0
2020-04-12,0.0,0.0,1.0,0.0,4.0,1.0,1.0,6.0,1.0,7.0
2020-04-13,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,2.0
2020-04-14,0.0,0.0,2.0,0.0,1.0,6.0,1.0,8.0,2.0,10.0
...,...,...,...,...,...,...,...,...,...,...
2021-03-04,1.0,3.0,12.0,26.0,36.0,39.0,17.0,118.0,16.0,134.0
2021-03-05,3.0,1.0,9.0,14.0,24.0,28.0,11.0,77.0,13.0,90.0
2021-03-06,4.0,0.0,7.0,16.0,23.0,21.0,9.0,69.0,11.0,80.0


In [8]:
df_muertes_diarias.diff(1).drop(df_muertes_diarias.head(1).index, inplace=True)

,<=39,40-49,50-59,60-69,70-79,80-89,>=90,60 o mas,Menores de 60,Totales
date,,,,,,,,,,
2020-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-10,0.0,1.0,1.0,1.0,1.0,2.0,2.0,6.0,2.0,8.0
2020-04-11,1.0,0.0,0.0,2.0,2.0,3.0,0.0,7.0,1.0,8.0
2020-04-12,0.0,0.0,1.0,0.0,4.0,1.0,1.0,6.0,1.0,7.0
2020-04-13,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...
2021-03-12,1.0,1.0,5.0,20.0,22.0,30.0,10.0,82.0,7.0,89.0
2021-03-13,3.0,6.0,11.0,23.0,38.0,24.0,18.0,103.0,20.0,123.0
2021-03-14,4.0,6.0,5.0,14.0,37.0,25.0,9.0,85.0,15.0,100.0


In [9]:
df_muertes_diarias.set_index('date', inplace=True)